In [3]:
#quantum fourrier transform (qft) function


import cirq
from cirq.circuits import InsertStrategy
from cirq import H, SWAP, CZPowGate 
import cirq

def qft(n,qubits,circuit):
    
    #For each qubit
    for i in range(n):
        #Apply Hadamard to the qubit
        circuit.append(H(qubits[i]), strategy = InsertStrategy.NEW)
        #The InsertStrategy enum defines different strategies for inserting gates into a quantum circuit. 
        #The InsertStrategy.NEW strategy tells the circuit to insert the gate at the end of the circuit, 
        #regardless of where other gates are located
        
        #Apply CR_k gates where j is the control and i is the target
        k=2 #We start with k=2
        t=2/2**(k)
        for j in range(i+1,n):
            #Define and apply CR_k gate 
            cirq.CZPowGate(exponent=t)
            crk = CZPowGate(exponent = 2/2**(k))
            circuit.append(crk(qubits[j],qubits[i]))
            k=k+1 #Increment k at each step
 
        #Swap the qubits
    for i in range(n//2):
        circuit.append(SWAP(qubits[i],qubits[n-i-1]), strategy = InsertStrategy.NEW)

In [4]:
# This is how you input states to the QFT circuit

# n=3 

# inputs = ['000','001','010','011','100','101','110','111']

# for input in inputs:

#     qlist=cirq.LineQubit.range(n)
#     circuit= cirq.Circuit()
#     if input[0]=='1':
#         circuit.append(cirq.X(qlist[0]))    
#     if input[1]=='1':
#         circuit.append(cirq.X(qlist[1]))   
#     if input[2]=='1':
#         circuit.append(cirq.X(qlist[2]))
#     qft(3,qlist,circuit)
#     print(circuit)
#     s= cirq.Simulator()
#     result=s.simulate(circuit)
#     print("Res:",result)
#     print("For input:",input)
#     print('QFT of {x} is {y}'.format(x=input, y=result))


In [5]:
#Inverse QFT Function

def iqft(n,qubits,circuit):
    #Swap
    for i in range(n//2):
        circuit.append(SWAP(qubits[i],qubits[n-i-1]), strategy = InsertStrategy.NEW) 
    for i in range(n-1,-1,-1):
        #Apply CR_k gates where j is the control and i is the target
        k=n-i #We start with k=n-i
        for j in range(n-1,i,-1):
            #Define and apply CR_k gate
            crk = CZPowGate(exponent = -2/2**(k))
            circuit.append(crk(qubits[j],qubits[i]),strategy = InsertStrategy.NEW)
            k=k-1
        #Apply Hadamard to the qubit
        circuit.append(H(qubits[i]), strategy = InsertStrategy.NEW)

In [6]:
#Function implementing QPE

def qpe(t,control, target, circuit, CU):
    circuit.append(H.on_each(control))
    #Apply Hadamard to control qubits
    for i in range(t):
        CUi = CU**(2**i)
        circuit.append(CUi(control[t-i-1],*target))
    #Apply CU gates
    iqft(t, control, circuit)  
    #Apply inverse QFT

In [14]:
from cirq import CZPowGate
phase = 3/16  
CU = CZPowGate(exponent=phase*2)#We need a 2 since CZPowGate adds a phase of e^{\pi i \phi}.

In [15]:
import cirq
from cirq import X
from cirq.contrib.svg import SVGCircuit

#Create cirucit
circuit = cirq.Circuit()
print(type(circuit))
t=4 #Number of qubits in the control register
#This depends on the precision on the phase you need 
n=1 #Number of qubits in the register storing eigenvector

#Create t control qubits
control = [cirq.LineQubit(i) for i in range(t) ]

#Create n target qubits
target = [cirq.LineQubit(i) for i in range(t,t+n) ]

#Set target qubit to state |1> 
circuit.append(X.on_each(target))

#Apply QPE

qpe(t,control, target, circuit, CU)
circuit.append(cirq.measure(*control, key='result')) 

#The key='result' argument in the line circuit.append(cirq.measure(*control, key='result')) tells 
#the quantum circuit to store the measurement results in a variable called result.
s=cirq.Simulator()
print('Sample the circuit:')
samples=s.run(circuit, repetitions=1000)

# Print a histogram of results
print(samples.histogram(key='result'))
print(circuit)

<class 'cirq.circuits.circuit.Circuit'>
Sample the circuit:
Counter({3: 1000})
0: ───H───────────────────────────────@───×───────────────────────────────────────────────@──────────@─────────@────────H───M('result')───
                                      │   │                                               │          │         │            │
1: ───H──────────────────────@────────┼───┼───×────────────────────@─────────@────────H───┼──────────┼─────────@^-0.5───────M─────────────
                             │        │   │   │                    │         │            │          │                      │
2: ───H─────────────@────────┼────────┼───┼───×───────@────────H───┼─────────@^-0.5───────┼──────────@^-0.25────────────────M─────────────
                    │        │        │   │           │            │                      │                                 │
3: ───H───@─────────┼────────┼────────┼───×───────H───@^-0.5───────@^-0.25────────────────@^(-1/8)──────────────────────────M─

The phase can thus be calculated to be $\phi= \frac{3}{2^{4}} = \frac{3}{16}$ (since $t=4$). This is exactly as expected.

In [16]:
import numpy as np
array = np.array([[1, 0, 0, 0],[0, 1, 0, 0],[0, 0, 1, 0],[0, 0, 0, -0.4762382+0.87931631j]])

U = cirq.MatrixGate(array)  #the U matrix is given here, we have to estimate the phase
CU = U.controlled()

In [21]:
import cirq

n=2
for t in range (1,10):
    #Create cirucit
    circuit = cirq.Circuit()
#Create t control qubits
    control = [cirq.LineQubit(i) for i in range(t) ]
#Create n target qubits
    target = [cirq.LineQubit(i) for i in range(t,t+n) ]
#Set target qubit to state |1> 
    circuit.append(X.on_each(target))
#Apply QPE
    qpe(t,control, target, circuit, CU)
    circuit.append(cirq.measure(*control, key='result')) 
#The key='result' argument in the line circuit.append(cirq.measure(*control, key='result')) tells 
#the quantum circuit to store the measurement results in a variable called result.
    s=cirq.Simulator()
#     print('Sample the circuit:')
    samples=s.run(circuit, repetitions=1000)
# Print a histogram of results
    print(samples.histogram(key='result'), t)
#     print(circuit)

Counter({1: 735, 0: 265}) 1
Counter({1: 731, 2: 161, 3: 55, 0: 53}) 2
Counter({3: 631, 2: 212, 4: 53, 1: 38, 5: 27, 0: 16, 7: 13, 6: 10}) 3
Counter({5: 804, 6: 102, 4: 33, 7: 17, 8: 8, 1: 6, 3: 6, 12: 6, 2: 5, 9: 4, 0: 3, 11: 2, 14: 1, 10: 1, 15: 1, 13: 1}) 4
Counter({11: 458, 10: 368, 9: 46, 12: 38, 8: 19, 13: 16, 7: 6, 15: 6, 14: 6, 6: 5, 5: 4, 19: 3, 2: 3, 17: 3, 16: 2, 20: 2, 1: 2, 24: 2, 22: 2, 23: 2, 4: 2, 0: 1, 25: 1, 29: 1, 18: 1, 3: 1}) 5
Counter({21: 993, 20: 3, 23: 2, 18: 1, 22: 1}) 6
Counter({42: 963, 43: 19, 41: 9, 44: 5, 39: 1, 45: 1, 38: 1, 40: 1}) 7
Counter({84: 859, 85: 58, 83: 25, 86: 10, 82: 9, 87: 7, 80: 6, 88: 3, 89: 3, 77: 2, 61: 1, 90: 1, 56: 1, 91: 1, 49: 1, 128: 1, 63: 1, 73: 1, 93: 1, 145: 1, 58: 1, 70: 1, 81: 1, 194: 1, 99: 1, 79: 1, 76: 1, 78: 1}) 8
Counter({168: 521, 169: 306, 167: 44, 170: 40, 171: 13, 166: 9, 165: 9, 172: 8, 174: 5, 173: 5, 164: 4, 158: 3, 176: 2, 156: 2, 163: 2, 162: 2, 175: 2, 178: 2, 187: 1, 177: 1, 181: 1, 343: 1, 161: 1, 452: 1, 140:

The estimated phase here is $\phi_{6}= \frac{21}{2^6} = 0.328125 $. The reason we chose $t=6$ is because beyond that $t$ the precision of the phase estimated is almost the same. Hence this is the most precise phase the $QPE$ algorithm can give us. $21$ comes from the fact that most probable measurement values for $t=6$ circuit is $21$.